**실습 목표**

💡 딥러닝 모델 (MLP, CNN, VGG19)을 이용한 개 vs.고양이 이미지 분류

💡 모델이 이미지의 어떤 특징을 추출하였는지 시각화 

**필요 라이브러리 import 및 추가된 데이터 path 확인**

In [ ]:
import numpy as np 
import pandas as pd 
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**zip파일 압축 해제**

In [ ]:
from zipfile import ZipFile

base_path = '/kaggle/input/dogs-vs-cats/'
unzip_path = '/kaggle/working/' # 현재 디렉토리

# 반복문 이용
for folder in os.listdir(path = base_path):
    if folder.split(".")[1] == 'zip':
        with ZipFile(base_path + folder, 'r') as zipfile:
            zipfile.extractall(unzip_path)
            print(f'{folder} 압축해제 완료!')

### 데이터분석 및 전처리

모델에 입력하여 컴퓨터가 학습 할 수 있도록 전처리 해준다. 

In [ ]:
train_dir = os.path.join(unzip_path, 'train')
test_dir = os.path.join(unzip_path, 'test1')

# 파일리스트 가져오기
train_img_names = os.listdir(train_dir)
test_img_names = os.listdir(test_dir)

print('total training images : ', len(train_img_names))
print('total test images : ', len(test_img_names))

In [ ]:
train_img_names[:5]

**이미지 이름과 분류 카테고리 정보를 갖는 dataframe 생성**

In [ ]:
categories = list()

# 반복문 이용
for image in train_img_names:
    category = image.split('.')[0]
    if category == 'dog':
        categories.append('dog')
    else:
        categories.append('cat')
        
df = pd.DataFrame({'Image':train_img_names, 'Category':categories})[:2000]

**📌Try it!!**

In [ ]:
# 만들어진 데이터프레임 df를 출력해보자.


**데이터 시각화**

In [ ]:
# 필요 라이브러리 import
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
f = plt.figure(figsize=(5, 5))
ax = f.add_subplot()
sns.countplot(data=df, x='Category', ax=ax)

for patch in ax.patches:
    label_x = patch.get_x() + patch.get_width()/2
    label_y = patch.get_y() + patch.get_height()/2
    text_msg = str(int(patch.get_height())) 
    ax.text(label_x, label_y, text_msg, horizontalalignment='center', verticalalignment='center')
    
plt.show()

random 함수를 이용하여 이미지 확인하기

In [ ]:
import random

sample = random.choice(train_img_names)
sample_path = '/kaggle/working/train/' + sample
plt.imshow(plt.imread(sample_path))
plt.show()

**📌 Try it!!**

실행할때마다 정말로 random하게 이미지가 출력되는지 확인해보자.

In [ ]:
# 한번에 여러개의 이미지를 출력
path = '/kaggle/working/train/'
plt.figure(figsize=(10,10))
for i in range(25):
    img_path = path + df.Image[i]
    
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(plt.imread(img_path))
    plt.xlabel(df.Category[i])
plt.show()

### 훈련데이터, 테스트데이터 나누기

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.25, stratify=df['Category'])
# dataframe의 기존 인덱스 제거
train = train.reset_index(drop=True)
# test = test.reset_index(drop=False)

🎁 **Try it !!!**

train, test 데이터를 각각 reset_index(drop=True) 처리를 했을 때와

하지 않았을 때 데이터프레임 인덱스 차이를 확인하여 보세요.

**훈련데이터와 테스트데이터 갯수 시각화**

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(13,5))
sns.countplot(data=train, x='Category', palette='magma', ax=ax[0])
sns.countplot(data=test, x='Category', palette='magma', ax=ax[1])

for patch in ax[0].patches:
    label_x = patch.get_x() + patch.get_width()/2
    label_y = patch.get_y() + patch.get_height()/2
    text_msg = str(int(patch.get_height())) 
    ax[0].text(label_x, label_y, text_msg, horizontalalignment='center', verticalalignment='center')

for patch in ax[1].patches:
    label_x = patch.get_x() + patch.get_width()/2
    label_y = patch.get_y() + patch.get_height()/2
    text_msg = str(int(patch.get_height())) 
    ax[1].text(label_x, label_y, text_msg, horizontalalignment='center', verticalalignment='center')

plt.show()

In [ ]:
# keras의 ImageDataGenerator 라이브러리를 이용하여 이미지 전처리 진행
from tensorflow.keras.preprocessing.image import ImageDataGenerator

height, width, channel = (150, 150, 3)

train_datagen = ImageDataGenerator(rescale=1. / 255.)

train_generator = train_datagen.flow_from_dataframe(train,
                                                   directory = './train',
                                                   x_col='Image',
                                                   y_col='Category',
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   color_mode= 'rgb',
                                                   target_size=(height, width))

test_datagen = ImageDataGenerator(rescale=1. / 255.)
test_generator = test_datagen.flow_from_dataframe(test,
                                                   directory = './train',
                                                   x_col='Image',
                                                   y_col='Category',
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   color_mode= 'rgb',
                                                   target_size=(height, width))

### 모델 훈련

In [ ]:
# 필요 라이브러리 import
import tensorflow

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, InputLayer, Resizing
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD

### MLP

In [ ]:
# MLP 모델 정의
mlp_model = Sequential()

mlp_model.add(InputLayer((height, width, channel)))
mlp_model.add(Resizing(48, 48, interpolation='bilinear'))
mlp_model.add(Flatten()) 
mlp_model.add(Dense(2048, activation='relu'))
mlp_model.add(Dense(1024, activation='relu'))
mlp_model.add(Dense(512, activation='relu'))
mlp_model.add(Dense(128, activation='relu'))
mlp_model.add(Dense(2, activation='softmax'))

In [ ]:
# 모델 확인
mlp_model.summary()

In [ ]:
# 모델 컴파일
mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

# 모델 훈련
mlp_history = mlp_model.fit(train_generator,
                           validation_data=test_generator,
                           epochs=20
                           )

In [ ]:
# 정확도와 손실값 확인

mlp_acc = mlp_history.history['accuracy']
mlp_val_acc = mlp_history.history['val_accuracy']
mlp_loss = mlp_history.history['loss']
mlp_val_loss = mlp_history.history['val_loss']

mlp_epochs = range(len(mlp_acc))

plt.plot(mlp_epochs, mlp_acc, 'r', label='Training accuracy')
plt.plot(mlp_epochs, mlp_val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(mlp_epochs, mlp_loss, 'r', label='Training Loss')
plt.plot(mlp_epochs, mlp_val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### ✅ 모델이 예측한 결과 시각화

In [ ]:
class_names = ['cat','dogs']
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    true_label = np.argmax(true_label)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(2), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)
    true_label = np.argmax(true_label)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
x, y = test_generator.next()
predictions = mlp_model.predict(x)

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(4*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, y, x)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions, y)
plt.show()

### CNN

In [ ]:
# CNN 모델 정의

cnn_model = Sequential()

cnn_model.add(Conv2D(filters=32, kernel_size=3,activation="relu", input_shape=(height, width, channel)))
cnn_model.add(MaxPool2D(pool_size=2, strides=2))
cnn_model.add(Conv2D(filters=64, kernel_size=3,activation="relu"))
cnn_model.add(MaxPool2D(pool_size=2, strides=2))
cnn_model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
cnn_model.add(MaxPool2D(pool_size=2, strides=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(units=512, activation="relu"))
cnn_model.add(Dropout(0.15))
cnn_model.add(Dense(units=2, activation="softmax"))

In [ ]:
# 모델 확인
cnn_model.summary()

In [ ]:
# 모델 컴파일
cnn_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 모델 훈련
cnn_history = cnn_model.fit(train_generator,
                            validation_data=test_generator,
                            epochs=20)

In [ ]:
# 모델 저장
cnn_model.save('cnn_model.h5')

In [ ]:
# 정확도와 손실값 확인

cnn_acc = cnn_history.history['accuracy']
cnn_val_acc = cnn_history.history['val_accuracy']
cnn_loss = cnn_history.history['loss']
cnn_val_loss = cnn_history.history['val_loss']

cnn_epochs = range(len(cnn_acc))

plt.plot(cnn_epochs, cnn_acc, 'r', label='Training accuracy')
plt.plot(cnn_epochs, cnn_val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(cnn_epochs, cnn_loss, 'r', label='Training Loss')
plt.plot(cnn_epochs, cnn_val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### ✅ CNN 모델이 예측한 결과 시각화

In [ ]:
predictions = cnn_model.predict(x)

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(4*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, y, x)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions, y)
plt.show()

### VGG19

In [ ]:
# VGG 모델 정의
img_input = Input(shape = (height,width,channel))

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block1_pool')(x)
x = BatchNormalization()(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block2_pool')(x)
x = BatchNormalization()(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block3_pool')(x)
x = BatchNormalization()(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block4_pool')(x)
x = BatchNormalization()(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block5_pool')(x)
x = BatchNormalization()(x)

x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(2, activation='softmax', name='predictions')(x)

vgg_model = Model(img_input, x, name='vgg16')

In [ ]:
# 모델 확인
vgg_model.summary()

In [ ]:
# 모델 컴파일
vgg_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

# 모델 학습
vgg_history = vgg_model.fit(train_generator,
                            validation_data=test_generator,
                            epochs=20)

In [ ]:
# 정확도 및 손실값 확인

vgg_acc = vgg_history.history['accuracy']
vgg_val_acc = vgg_history.history['val_accuracy']
vgg_loss = vgg_history.history['loss']
vgg_val_loss = vgg_history.history['val_loss']

vgg_epochs = range(len(vgg_acc))

plt.plot(vgg_epochs, vgg_acc, 'r', label='Training accuracy')
plt.plot(vgg_epochs, vgg_val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(vgg_epochs, vgg_loss, 'r', label='Training Loss')
plt.plot(vgg_epochs, vgg_val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### ✅ VGG 모델이 예측한 결과 시각화

In [ ]:
x, y = test_generator.next()
predictions = vgg_model.predict(x)

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(4*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, y, x)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions, y)
plt.show()

### 모델이 어떠한 특징으로 이미지를 분류 한 걸까!?

👀시각화 해보자! 

In [ ]:
# 저장한 cnn 모델 불러오기
from tensorflow.keras.models import load_model
saved_model = load_model('cnn_model.h5')

**🎁Try it!!**

불러온 모델 정보를 표시해보자.

In [ ]:
# 불러온 모델 정보 확인


**훈련에 포함되지 않은 이미지 하나 가져오기**

In [ ]:
# 리스트 형태이기 때문에 head()사용할 수 없으므로 [:5]로 확인
test_img_names[:5]

In [ ]:
from  tensorflow.keras.preprocessing import image

img_path = './test1/8961.jpg'
# 모델이 훈련될 때 입력에 적용한 전처리 방식을 동일하게 사용해줘야합니다. (color정보, 이미지 크기 등)

img = image.load_img(img_path, target_size=(150, 150), color_mode='rgb')
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

# 이미지 크기 (1, 150, 150, 3)
print(img_tensor.shape)

In [ ]:
# 불러온 이미지 확인
plt.imshow(img_tensor[0])
plt.show()

모델이 사진을보고 각 층별로 어떤 특징을 추출했는지 살펴보자.

In [ ]:
from tensorflow.keras import models


# 상위 8개 층의 출력을 추출합
layer_outputs = [layer.output for layer in saved_model.layers[:8]]
# 입력에 대해 8개 층의 출력을 반환하는 모델을 생성
activation_model = models.Model(inputs=saved_model.input, outputs=layer_outputs)

In [ ]:
# 층의 활성화마다 하나씩 8개의 넘파이 배열로 이루어진 리스트를 반환합니다:
activations = activation_model.predict(img_tensor)

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 31], cmap='viridis')
plt.show()


다른 채널도 그려보자.

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 9], cmap='viridis')
plt.show()

채널 수를 바꾸어 가며 확인해보자

강아지 얼굴 무늬 중 색이 구분되는 선이 뚜렷히 관찰된다.

이제 네트워크의 모든 활성화층을 시각화 해봅시다.

**특성맵 그리기**

In [ ]:
# 층의 이름을 그래프 제목으로 사용
layer_names = []
for layer in saved_model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16

# 특성 맵 그리기
for layer_name, layer_activation in zip(layer_names, activations):
    # 특성 맵에 있는 특성의 수
    n_features = layer_activation.shape[-1]

    # 특성 맵의 크기 : (1, size, size, n_features)
    size = layer_activation.shape[1]

    # 활성화 채널을 위한 그리드 크기
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # 각 활성화를 하나의 큰 그리드에 채웁니다
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # 그래프로 나타내기 좋게 특성을 처리합니다
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # 그리드 출력
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()